In [6]:
import glob
import time

import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip

import sklearn
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label

%matplotlib inline

-----------------------
None
ffmpeg -version
-----------------------


In [7]:
from src.features import extract_features, get_hog_features
from src.utils import *
from src.search import Search

In [8]:
non_vehicles_images_names = glob.glob("data/non-vehicles/non-vehicles/**/*.png")
vehicles_images_names = glob.glob("data/vehicles/vehicles/**/*.png")    

In [14]:
configs = [("RGB", 9, 8, 2),
           ("HLS", 9, 8, 2),
           ("HSV", 9, 8, 2),
           ("YUV", 11, 8, 2),
           ("LUV", 9, 8, 2),
           ("YCrCb", 9, 8, 2),
           ("YCrCb", 11, 8, 2),
           ("YUV", 11, 16, 2)]

header_line = ('Colorspace | Orientations | Pixels Per Cell | Cells Per Block | Evaluation time ' +
    '| Accuracy')
divider = '--- | --- | --- | --- | --- | ---'
table_lines = [header_line, divider]
for config in configs:
    vehicle_features = extract_features(
        vehicles_images_names,
        orient=config[1],
        pix_per_cell=config[2],
        cell_per_block=config[3],
        hog_channel="ALL",
        cspace=config[0]
    )
    non_vehicles_features = extract_features(
        non_vehicles_images_names,
        orient=config[1],
        pix_per_cell=config[2],
        cell_per_block=config[3],
        hog_channel="ALL",
        cspace=config[0]
    )

    X = np.vstack((vehicle_features, non_vehicles_features)).astype(np.float64)
    Y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicles_features))))

    train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size=0.2, random_state=42)

    svc = LinearSVC()
    t = time.time()
    svc.fit(train_features, train_labels)
    t2 = time.time()
    
    evaluation_time = round(t2 - t, 2)
    accuracy = round(svc.score(test_features, test_labels), 4)
    
    print("{} {} {} {}".format(config[0], config[1], config[2], config[3]))
    print('Test Accuracy of SVC = ', accuracy)
    print(evaluation_time, 'Seconds to train SVC...')
    print()
    
    line = "{} | {} | {} | {} | {} | {}".format(
        config[0],
        config[1],
        config[2],
        config[3],
        evaluation_time,
        accuracy
    )
    table_lines.append(line)

RGB 9 8 2
Test Accuracy of SVC =  0.9758
11.07 Seconds to train SVC...

HLS 9 8 2
Test Accuracy of SVC =  0.9814
5.63 Seconds to train SVC...

HSV 9 8 2
Test Accuracy of SVC =  0.9828
6.33 Seconds to train SVC...

YUV 11 8 2
Test Accuracy of SVC =  0.9825
6.03 Seconds to train SVC...

LUV 9 8 2
Test Accuracy of SVC =  0.9764
6.61 Seconds to train SVC...

YCrCb 9 8 2
Test Accuracy of SVC =  0.9797
5.46 Seconds to train SVC...

YCrCb 11 8 2
Test Accuracy of SVC =  0.9823
5.93 Seconds to train SVC...

YUV 11 16 2
Test Accuracy of SVC =  0.9828
0.83 Seconds to train SVC...



In [15]:
print('For MD file:')
for line in table_lines:
    print(line)

For MD file:
Colorspace | Orientations | Pixels Per Cell | Cells Per Block | Evaluation time | Accuracy
--- | --- | --- | --- | --- | ---
RGB | 9 | 8 | 2 | 11.07 | 0.9758
HLS | 9 | 8 | 2 | 5.63 | 0.9814
HSV | 9 | 8 | 2 | 6.33 | 0.9828
YUV | 11 | 8 | 2 | 6.03 | 0.9825
LUV | 9 | 8 | 2 | 6.61 | 0.9764
YCrCb | 9 | 8 | 2 | 5.46 | 0.9797
YCrCb | 11 | 8 | 2 | 5.93 | 0.9823
YUV | 11 | 16 | 2 | 0.83 | 0.9828
